In [1]:
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm


In [5]:
# **Dataset Class with Preloading**
class PreloadedDataset(Dataset):
    def __init__(self, root_dir, categories, sequence_length=8, transform=None):
        """
        Args:
            root_dir (str): Root directory containing category folders.
            categories (list): List of category names (subfolder names).
            sequence_length (int): Number of consecutive frames in each sequence.
            transform (callable, optional): Transform to apply to each frame.
        """
        self.data = []
        self.labels = []
        self.sequence_length = sequence_length
        self.transform = transform

        for label, category in enumerate(categories):
            category_path = os.path.join(root_dir, category)
            if not os.path.exists(category_path):
                print(f"Category folder does not exist: {category_path}")
                continue

            # Load all PNGs into memory
            print(f"Preloading category: {category}")
            frame_files = sorted([f for f in os.listdir(category_path) if f.endswith(".png")])
            for file in frame_files:
                img = Image.open(os.path.join(category_path, file)).convert("RGB")
                if self.transform:
                    img = self.transform(img)
                self.data.append(img)  # Add preprocessed image
                self.labels.append(label)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Create sequences on-the-fly from preloaded data
        start_idx = idx
        end_idx = start_idx + self.sequence_length

        # Handle edge cases by padding with zeros
        if end_idx > len(self.data):
            sequence = self.data[start_idx:] + [torch.zeros_like(self.data[0])] * (end_idx - len(self.data))
        else:
            sequence = self.data[start_idx:end_idx]

        # Stack into tensor of shape (C, T, H, W)
        sequence = torch.stack(sequence, dim=1)
        label = self.labels[idx]
        return sequence, label

In [2]:
class Optimized3DCNN(nn.Module):
    def __init__(self, num_classes):
        super(Optimized3DCNN, self).__init__()
        self.conv1 = nn.Conv3d(3, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm3d(32)
        self.pool1 = nn.MaxPool3d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv3d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm3d(64)
        self.pool2 = nn.MaxPool3d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv3d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm3d(128)
        self.pool3 = nn.MaxPool3d(kernel_size=2, stride=2)

        self.global_pool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        x = self.global_pool(x)
        x = x.view(x.size(0), -1)  # Flatten for the FC layer
        x = self.fc(x)
        return x


In [11]:
# **Prepare Datasets and DataLoaders**
categories = ["Abuse", "Arson", "Assault", "Burglary", "Explosion", "Fighting",
              "NormalVideos", "RoadAccidents", "Robbery", "Shooting", "Shoplifting",
              "Stealing", "Vandalism"]

train_root = "Train"
test_root = "Test"

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),  # Slight rotations to avoid distortion
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.Resize((112, 112)),  # Ensure the size is consistent
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


In [ ]:

train_dataset = PreloadedDataset(root_dir=train_root, categories=categories, sequence_length=16, transform=transform)
#

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)  # Preloading, so no workers
#

Preloading category: Abuse
Preloading category: Arson
Preloading category: Assault
Preloading category: Burglary
Preloading category: Explosion
Preloading category: Fighting
Preloading category: NormalVideos
Preloading category: RoadAccidents
Preloading category: Robbery
Preloading category: Shooting
Preloading category: Shoplifting
Preloading category: Stealing
Preloading category: Vandalism


In [18]:
test_dataset = PreloadedDataset(root_dir=test_root, categories=categories, sequence_length=8, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

Preloading category: Abuse
Preloading category: Arson
Preloading category: Assault
Preloading category: Burglary
Preloading category: Explosion
Preloading category: Fighting
Preloading category: NormalVideos
Preloading category: RoadAccidents
Preloading category: Robbery
Preloading category: Shooting
Preloading category: Shoplifting
Preloading category: Stealing
Preloading category: Vandalism


In [ ]:
# **Model Setup**
num_classes = len(categories)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True
model = Optimized3DCNN(num_classes=num_classes).to(device)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

optimizer = optim.AdamW(model.parameters(), lr=0.0005, weight_decay=1e-4)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

scaler = GradScaler()  # Mixed precision scaler


C:\Users\adars\AppData\Local\Temp\ipykernel_20868\3758203713.py:13: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()  # Mixed precision scaler


In [21]:
# **Training Loop**
num_epochs = 5
save_dir = "models"  # Folder to save models
os.makedirs(save_dir, exist_ok=True)

In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F  # Import this for functional operations
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.cuda.amp import GradScaler, autocast


In [15]:
for epoch in range(num_epochs):
    print(f"Starting Epoch {epoch + 1}/{num_epochs}")
    model.train()
    running_loss = 0.0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}")

    for batch_idx, (inputs, labels) in enumerate(progress_bar):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        # Mixed precision training
        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        # Backward pass
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()

        # Update progress bar
        progress_bar.set_postfix(loss=loss.item())

        # Debug every 10 batches
        if batch_idx % 1000 == 0:
            print(f"Batch {batch_idx + 1}/{len(train_loader)}, Loss: {loss.item():.4f}")

    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs} completed with Average Loss: {epoch_loss:.4f}")

    # Save model checkpoint
    torch.save(model.state_dict(), os.path.join(save_dir, f"simple_3dcnn_epoch_{epoch + 1}.pth"))
    print(f"Model saved for Epoch {epoch + 1}")

    # Step scheduler
    scheduler.step()

print("Training complete.")

Starting Epoch 1/5


Epoch 1/5:   0%|          | 0/10648 [00:00<?, ?it/s]C:\Users\adars\AppData\Local\Temp\ipykernel_20868\1675048004.py:12: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1/5:   0%|          | 0/10648 [00:00<?, ?it/s]


NameError: name 'F' is not defined

In [23]:
for epoch in range(num_epochs):
    print(f"Starting Epoch {epoch + 1}/{num_epochs}")
    
    # Training Phase
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        # Forward pass and loss calculation
        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        # Backward pass
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()

        # Calculate training accuracy
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    # Calculate average training loss and accuracy
    train_loss = running_loss / len(train_loader)
    train_accuracy = correct / total
    print(f"Epoch {epoch + 1}/{num_epochs}: Train Loss = {train_loss:.4f}, Train Accuracy = {train_accuracy:.4f}")

    # Save model checkpoint (optional)
    torch.save(model.state_dict(), os.path.join(save_dir, f"simple_3dcnn_epoch_{epoch + 1}.pth"))

    # Step the learning rate scheduler
    scheduler.step()

print("Training complete.")


Starting Epoch 1/5


C:\Users\adars\AppData\Local\Temp\ipykernel_20868\2143657934.py:14: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1/5: Train Loss = 1.3708, Train Accuracy = 0.6864
Starting Epoch 2/5
Epoch 2/5: Train Loss = 0.9059, Train Accuracy = 0.8947
Starting Epoch 3/5
Epoch 3/5: Train Loss = 0.7595, Train Accuracy = 0.9550
Starting Epoch 4/5
Epoch 4/5: Train Loss = 0.6905, Train Accuracy = 0.9795
Starting Epoch 5/5
Epoch 5/5: Train Loss = 0.6552, Train Accuracy = 0.9894
Training complete.


MODEL EVALUATION

In [24]:
# Test Phase
model.eval()
test_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        # Calculate accuracy
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

# Calculate average test loss and accuracy
test_loss /= len(test_loader)
test_accuracy = correct / total
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


Test Loss: 2.3720, Test Accuracy: 0.3271


In [25]:
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


Test Accuracy: 32.71%


LOADING MODEL


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F






# Define categories and device
categories = ["Abuse", "Arson", "Assault", "Burglary", "Explosion", "Fighting",
              "NormalVideos", "RoadAccidents", "Robbery", "Shooting", "Shoplifting",
              "Stealing", "Vandalism"]
num_classes = len(categories)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the model
model = Optimized3DCNN(num_classes=num_classes).to(device)

# Path to the saved model
saved_model_path = "models_with_additional_layers\simple_3dcnn_epoch_5.pth"  # Replace with your actual saved file name

# Load the saved state dictionary
model.load_state_dict(torch.load(saved_model_path))

# Set the model to evaluation mode
model.eval()

print("Model loaded successfully and set to evaluation mode.")


Model loaded successfully and set to evaluation mode.


<>:21: SyntaxWarning: invalid escape sequence '\s'
<>:21: SyntaxWarning: invalid escape sequence '\s'
C:\Users\adars\AppData\Local\Temp\ipykernel_18272\1257002490.py:21: SyntaxWarning: invalid escape sequence '\s'
  saved_model_path = "models_with_additional_layers\simple_3dcnn_epoch_5.pth"  # Replace with your actual saved file name
C:\Users\adars\AppData\Local\Temp\ipykernel_18272\1257002490.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly

VIDEO PROCESSING PIPELINE

In [6]:
import os
import cv2
import torch
import torchvision.transforms as transforms
from PIL import Image
import torch.nn.functional as F

In [ ]:
transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [16]:
# Function to extract frames from a video
def extract_frames(video_path, sequence_length=8):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = Image.fromarray(frame)  # Convert frame to PIL Image
        frame = transform(frame)       # Apply defined transform
        frames.append(frame)
    cap.release()

    # Split frames into sequences
    frame_sequences = []
    for i in range(0, len(frames) - sequence_length + 1, sequence_length):
        frame_sequences.append(torch.stack(frames[i:i + sequence_length], dim=1))  # Shape: (C, T, H, W)

    return frame_sequences


In [17]:
model = Optimized3DCNN(num_classes=13)
model.load_state_dict(torch.load("models_with_additional_layers\simple_3dcnn_epoch_5.pth"))  # Update with your saved model path
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
C:\Users\adars\AppData\Local\Temp\ipykernel_18272\2884218857.py:2: SyntaxWarning: invalid escape sequence '\s'
  model.load_state_dict(torch.load("models_with_additional_layers\simple_3dcnn_epoch_5.pth"))  # Update with your saved model path
C:\Users\adars\AppData\Local\Temp\ipykernel_18272\2884218857.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are expli

In [18]:
# Prediction function
def predict_video(video_path):
    frame_sequences = extract_frames(video_path, sequence_length=16)
    predictions = []
    with torch.no_grad():
        for frame_sequence in frame_sequences:
            frame_sequence = frame_sequence.unsqueeze(0).to(device)  # Add batch dimension
            outputs = model(frame_sequence)
            _, predicted = torch.max(outputs, 1)
            predictions.append(predicted.item())
    # Return the most common predicted class
    return max(set(predictions), key=predictions.count)

In [22]:
# Run prediction
video_path = "output_folder_video\RoadAccidents017_x264.avi"  # Replace with your video path
predicted_class = predict_video(video_path)
print(f"Predicted Class: {predicted_class}")

Predicted Class: 4


<>:2: SyntaxWarning: invalid escape sequence '\R'
<>:2: SyntaxWarning: invalid escape sequence '\R'
C:\Users\adars\AppData\Local\Temp\ipykernel_18272\3703828210.py:2: SyntaxWarning: invalid escape sequence '\R'
  video_path = "output_folder_video\RoadAccidents017_x264.avi"  # Replace with your video path


In [23]:
categories = ["Abuse", "Arson", "Assault", "Burglary", "Explosion", "Fighting",
              "NormalVideos", "RoadAccidents", "Robbery", "Shooting", "Shoplifting",
              "Stealing", "Vandalism"]

print(f"Predicted Category: {categories[predicted_class]}")


Predicted Category: Explosion
